In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
from xdg.Mime import get_type2, MIMEtype

In [6]:
dir(get_type2('./scratchpad.ipynb'))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_comment',
 '_init',
 '_load',
 'canonical',
 'get_comment',
 'inherits_from',
 'media',
 'subtype']

In [7]:
from xdg.Menu import parse

In [24]:
for submenu in parse('/etc/xdg/menus/arch-applications.menu').Submenus:
   if submenu.Entries:
      first_entry = submenu.Entries[0]
      
      print(first_entry.DesktopEntry.getExec())
      
      for k in dir(first_entry):
         if not k.startswith('__'):
            print(f'{k}={getattr(first_entry, k)}')
      
      break

bluetooth-sendto
Add=False
Allocated=True
Categories=['GTK', 'GNOME', 'Utility', 'FileTools']
DesktopEntry=Bluetooth Transfer
DesktopFileID=bluetooth-sendto.desktop
Filename=bluetooth-sendto.desktop
MatchedInclude=True
Original=None
Parents=[<xdg.Menu.Menu object at 0x7f8b35ecf4a8>]
Prefix=
Show=NoDisplay
TYPE_BOTH=Both
TYPE_SYSTEM=System
TYPE_USER=User
_MenuEntry__setFilename=<bound method MenuEntry.__setFilename of bluetooth-sendto.desktop>
_key=<bound method MenuEntry._key of bluetooth-sendto.desktop>
getDir=<bound method MenuEntry.getDir of bluetooth-sendto.desktop>
getType=<bound method MenuEntry.getType of bluetooth-sendto.desktop>
save=<bound method MenuEntry.save of bluetooth-sendto.desktop>
setAttributes=<bound method MenuEntry.setAttributes of bluetooth-sendto.desktop>
updateAttributes=<bound method MenuEntry.updateAttributes of bluetooth-sendto.desktop>
